In [0]:
# ==========================================================
# UDV - ENTRENADORES
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from env_setup import *
from utils_liga1 import setup_adls, get_dbutils,get_entity_data,is_dataframe_empty,get_predecesor,get_pipeline_params, get_yaml_from_param,write_delta_udv, log,read_udv_table,get_pipeline
from pyspark.sql import SparkSession
from hm_entrenadores_equipo import carga_final_md_entrenadores,  carga_final_hm_entrenadores
import traceback
import sys

# ----------------------------------------------------------
# CONFIGURACIÓN INICIAL
# ----------------------------------------------------------
log("Inicio de ejecución del pipeline UDV", "INFO", "hm_entrenadores_equipo")

spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

In [0]:

# ----------------------------------------------------------
# PARÁMETROS Y PREDECESORES
# ----------------------------------------------------------
try:
    # Para pruebas: valor por defecto; en ADF se sobreescribe
    dbutils.widgets.text("prm_pipelineid", "12") 
    prm_pipelineid = int(dbutils.widgets.get("prm_pipelineid"))

    # Pipeline HM (este notebook está atado al HM)
    pipeline_name = get_pipeline(prm_pipelineid)
    entity_name = pipeline_name["pipeline"]               # ej: 'hm_entrenadores_equipo'
    parent_pipelineid = pipeline_name["parent_pipelineid"]

    # Pipeline MD (padre/hijo según tu modelo, igual que plantillas)
    pipeline_name_chd = get_pipeline(parent_pipelineid)
    entity_name_chd = pipeline_name_chd["pipeline"]       # ej: 'md_entrenadores'

    # Parámetros y predecesores del HM
    dict_params       = get_pipeline_params(prm_pipelineid)
    dict_predecesores = get_predecesor(prm_pipelineid)

    # Parámetros y predecesores del MD
    dict_params_chd        = get_pipeline_params(parent_pipelineid)
    dict_predecesores_chd  = get_predecesor(parent_pipelineid)

    # Predecesores UDV (para HM)
    prm_tabla_md_entrenadores      = dict_predecesores["md_entrenadores"]["RutaTabla"]
    prm_tabla_md_catalogo_equipos  = dict_predecesores["md_catalogo_equipos"]["RutaTabla"]
    
    entity_raw = dict_predecesores_chd["entrenadores"]["RutaTabla"]

    # Parámetros HM
    prm_capa_udv     = dict_params["CAPA_UDV"]
    prm_ruta_base    = dict_params["RUTA_BASE"]
    prm_ruta_tabla   = dict_params["RUTA_TABLA"]
    prm_formato      = dict_params["FORMATO_SALIDA"]
    prm_schema_tb    = dict_params["SCHEMA_TABLA"]    
    prm_tabla_output = dict_params["NOMBRE_TABLA"]      
    prm_ruta_yaml    = dict_params["YAML_PATH"]

    # Parámetros MD (desde pipeline padre/hijo)
    prm_capa_udv_chd     = dict_params_chd["CAPA_UDV"]
    prm_ruta_base_chd    = dict_params_chd["RUTA_BASE"]
    prm_ruta_tabla_chd   = dict_params_chd["RUTA_TABLA"]
    prm_formato_chd      = dict_params_chd["FORMATO_SALIDA"]
    prm_schema_tb_chd    = dict_params_chd["SCHEMA_TABLA"]  
    prm_tabla_output_chd = dict_params_chd["NOMBRE_TABLA"]  

    log("Parámetros cargados correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al cargar parámetros o predecesores: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:

# ----------------------------------------------------------
# LECTURA YAML
# ----------------------------------------------------------
try:
    yaml_conf = get_yaml_from_param(prm_ruta_yaml)

    prm_cols_raw_entrenadores_md   = yaml_conf[entity_name]["cols_raw_entrenadores_md"]
    prm_case_rules                 = yaml_conf[entity_name]["case_rules"]
    prm_rename_columns_md          = yaml_conf[entity_name]["rename_columns_md"]
    prm_schema_md                  = yaml_conf[entity_name]["schema_md"]

    prm_cols_catalogo_equipos_hm   = yaml_conf[entity_name]["cols_catalogo_equipos_hm"]
    prm_cols_raw_entrenadores_hm   = yaml_conf[entity_name]["cols_raw_entrenadores_hm"]
    prm_dedup_cols_hm              = yaml_conf[entity_name]["dedup_cols_hm"]
    prm_rename_columns_hm          = yaml_conf[entity_name]["rename_columns_hm"]
    prm_schema_hm                  = yaml_conf[entity_name]["schema_hm"]

    log("YAML cargado correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al leer YAML {prm_ruta_yaml}: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:

# ----------------------------------------------------------
#  LECTURA PREDECESORES (RAW)
# ----------------------------------------------------------
try:
    log("Lectura desde RAW (tbl_paths, flg_udv = 'N')", "INFO", entity_name_chd)

    # RAW entrenadores desde tbl_paths (flg_udv = 'N')
    df_raw_entrenadores = get_entity_data(entity_raw, dedup_cols=None)

    if is_dataframe_empty(df_raw_entrenadores):
        raise Exception(f"No se encontró data en RAW para la entidad: {entity_raw}")

    log("Lectura de RAW completada correctamente", "INFO", entity_name_chd)

except Exception as e:
    log(f"Error en lectura de RAW: {e}", "ERROR", entity_name_chd)
    print(traceback.format_exc())
    raise

log("Finalización de la etapa de lectura RAW", "INFO", entity_name_chd)

In [0]:
# ----------------------------------------------------------
# EJECUCIÓN MD_ENTRENADORES
# ----------------------------------------------------------
try:
    log("Inicio de Ejecución Principal - MD_ENTRENADORES", "INFO", entity_name_chd)

    df_final_md = carga_final_md_entrenadores(
        df_raw_entrenadores,
        prm_cols_raw_entrenadores_md,
        prm_case_rules,
        prm_rename_columns_md,
        prm_schema_md
    )


    # MERGE simple (UPSERT) sobre id_entrenador
    write_delta_udv(
        spark,
        df_final_md,
        prm_schema_tb_chd,         # schema MD (ej: tb_udv)
        prm_tabla_output_chd,      # md_entrenadores
        mode="merge",
        merge_condition="delta.id_entrenador = df.id_entrenador"
    )

    log("Proceso MD_ENTRENADORES completado correctamente", "SUCCESS", entity_name_chd)

except Exception as e:
    log(f"Error en ejecución principal MD_ENTRENADORES: {e}", "ERROR", entity_name_chd)
    print(traceback.format_exc())
    raise

log("Finalización del pipeline UDV - MD_ENTRENADORES", "INFO", entity_name_chd)


In [0]:

# ----------------------------------------------------------
# LECTURA PREDECESORES UDV (MDs) PARA HM
# ----------------------------------------------------------
try:
    log("Lectura de predecesores UDV (md_entrenadores, md_catalogo_equipos)", "INFO", entity_name)

    # 1) Catálogo de entrenadores (md_entrenadores) vía RutaTabla
    df_md_entrenadores = read_udv_table(prm_tabla_md_entrenadores)

    # 2) Catálogo de equipos (md_catalogo_equipos) vía RutaTabla
    df_md_catalogo_equipos = read_udv_table(prm_tabla_md_catalogo_equipos)

    if df_md_entrenadores is None or is_dataframe_empty(df_md_entrenadores):
        raise Exception("md_entrenadores está vacío o no existe.")

    if df_md_catalogo_equipos is None or is_dataframe_empty(df_md_catalogo_equipos):
        raise Exception("md_catalogo_equipos está vacío o no existe.")

    log("Predecesores UDV leídos correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error en lectura de predecesores UDV: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

log("Finalización de la etapa de lectura de predecesores UDV", "INFO", entity_name)


In [0]:

# ----------------------------------------------------------
# EJECUCIÓN HM_ENTRENADORES_EQUIPO
# ----------------------------------------------------------
try:
    log("Inicio de Ejecución Principal - HM_ENTRENADORES_EQUIPO", "INFO", entity_name)

    df_final_hm = carga_final_hm_entrenadores(
        df_md_catalogo_equipos,
        prm_cols_catalogo_equipos_hm,
        df_raw_entrenadores,
        prm_cols_raw_entrenadores_hm,
        df_md_entrenadores,
        prm_case_rules,
        prm_rename_columns_hm,
        prm_schema_hm,
        prm_dedup_cols_hm
    )

    if is_dataframe_empty(df_final_hm):
        log("df_final_hm vacío, no se realizará escritura en UDV", "WARN", entity_name)
    else:
        log("Escribiendo HM con overwrite dinámico por partición [periodo]", "INFO", entity_name)
    

        write_delta_udv(
            spark,
            df_final_hm,
            prm_schema_tb,          # schema HM (ej: tb_udv)
            prm_tabla_output,       # hm_entrenadores_equipo
            mode="overwrite",
            partition_by=["periodo"],
            overwrite_dynamic_partition=True
        )

        log("Escritura HM completada con overwrite dinámico", "SUCCESS", entity_name)

except Exception as e:
    log(f"Error en ejecución principal HM_ENTRENADORES_EQUIPO: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

log("Finalización del pipeline UDV - HM_ENTRENADORES_EQUIPO", "INFO", entity_name)